In [7]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.models import vgg16, resnet50

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
data_dir = 'Data/Training'

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageFolder(root=data_dir, transform=transform)

# DataLoader
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)

num_classes = len(os.listdir(data_dir))

# VGG16
model = resnet50()
model.fc = torch.nn.Linear(in_features=2048, out_features=num_classes)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 25
best_valid_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0

    # Training
    model.train()
    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            valid_loss += loss.item() * inputs.size(0)

    # Calculate average losses
    train_loss = train_loss / len(train_dataset)
    valid_loss = valid_loss / len(valid_dataset)

    print(f"Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")

    # Save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "Models/best_ResNet50_model.pth")

print("Training complete.")


Epoch 1/25: 100%|██████████| 286/286 [02:47<00:00,  1.71it/s]


Train Loss: 1.1199, Valid Loss: 0.7561


Epoch 2/25: 100%|██████████| 286/286 [02:49<00:00,  1.69it/s]


Train Loss: 0.8622, Valid Loss: 0.6443


Epoch 3/25: 100%|██████████| 286/286 [02:46<00:00,  1.71it/s]


...

Train Loss: 0.5354, Valid Loss: 1.4155


Epoch 6/25: 100%|██████████| 286/286 [02:53<00:00,  1.65it/s]


Train Loss: 0.4730, Valid Loss: 1.0185


Epoch 7/25:  65%|██████▌   | 187/286 [01:54<01:00,  1.63it/s]


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

test_dataset = ImageFolder(root='Data/Testing', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

def calculate_accuracy(loader, model, device):
    correct = 0
    total = 0
    model.eval()

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    return accuracy

test_accuracy = calculate_accuracy(test_loader, model, device)
print(f"Test Accuracy: {test_accuracy:.2f}%")

TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).